In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

In [8]:
username = ''
password = ''

tv = TvDatafeed(username, password)

ERROR:tvDatafeed.main:error while signin


In [9]:
df = tv.get_hist(symbol='ethusdt',exchange='BINANCE',interval=Interval.in_15_minute,n_bars=100000)
# data = data.iloc[:-96]

In [10]:
df.tail()

,symbol,open,high,low,close,volume
datetime,,,,,,
2024-09-16 10:15:00,BINANCE:ethusdt,2270.01,2276.04,2268.40,2272.67,6568.0908
2024-09-16 10:30:00,BINANCE:ethusdt,2272.67,2279.22,2272.23,2273.40,6367.7755
2024-09-16 10:45:00,BINANCE:ethusdt,2273.41,2278.98,2273.41,2278.38,4217.3971
2024-09-16 11:00:00,BINANCE:ethusdt,2278.38,2280.40,2276.10,2279.70,2925.6936
2024-09-16 11:15:00,BINANCE:ethusdt,2279.70,2282.55,2277.17,2280.20,1834.2687


In [11]:
def create_dataset(dataset, look_back=3):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

# simulated trading

In [12]:
i=11
df.iloc[:-96+i].copy().tail(1).index[0]

Timestamp('2024-09-15 14:00:00')

In [13]:
n=100*2
date = []
prediction_list = []
for i in range(n):
    data = df.iloc[:-n+i].copy()
    date.append(data.tail(1).index[0])
    X = data['close'].values
    X = X.astype('float32')
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X.reshape(-1, 1))

    train_size = int(len(X) * 0.9)
    test_size = len(X) - train_size
    train, test = X[0:train_size,:], X[train_size:len(X),:]

    look_back = 60
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    trainX = trainX.reshape(trainX.shape[0], 60)
    mod1 = XGBRegressor(learning_rate=0.1,random_seed=50)

    mod1.fit(trainX,trainY)

    prediction = mod1.predict(testX)
    prediction = scaler.inverse_transform(prediction.reshape(-1,1))
    prediction_list.append(prediction[-1][0])

In [14]:
simulated_trading = pd.DataFrame()
simulated_trading.index = np.array(date)
simulated_trading['open'] = df.loc[simulated_trading.index].open
simulated_trading['close'] = df.loc[simulated_trading.index].close
simulated_trading['prediction'] = prediction_list

simulated_trading['action'] = np.where(simulated_trading['open']>simulated_trading['prediction'],'sell','buy')
simulated_trading['gp'] = np.where(simulated_trading.action=='buy',(simulated_trading.close-simulated_trading.open)/simulated_trading.open,
                                   (simulated_trading.open-simulated_trading.close)/simulated_trading.close)


In [15]:
var =[]
for i in range(len(simulated_trading.gp.values)):
    var.append(np.quantile(simulated_trading.gp.iloc[:i+1],q=0.05))

cvar = []
for i in range(len(var)):
    cvar.append(np.mean(var[:i]))

In [16]:
simulated_trading['var'] = var
simulated_trading['cvar'] = cvar

In [24]:
#simulated_trading.to_csv('backtrack_simulation1.csv')

In [18]:
capital = 1000
for i in simulated_trading.gp:
    capital*=(1+i)
print('w/o any risk management:',capital)

w/o any risk management: 983.1054683451681


In [19]:
gp_var = []
for i in range(len(simulated_trading)):
    if simulated_trading.gp[i] > simulated_trading['var'][i]:
        gp_var.append(simulated_trading.gp[i])
    else:
        gp_var.append(simulated_trading['var'][i])

In [20]:
gp_cvar = []
for i in range(len(simulated_trading)):
    if simulated_trading.gp[i] > simulated_trading['cvar'][i]:
        gp_cvar.append(simulated_trading.gp[i])
    else:
        gp_cvar.append(simulated_trading['cvar'][i])

In [21]:
capital = 1000
for i in gp_var:
    capital*=(1+i)
print('w/ var',capital)

w/ var 1023.7962717240597


In [22]:
capital = 1000
for i in gp_cvar[1:]:
    capital*=(1+i)
print('w/ cvar',capital)

w/ cvar 1038.6764107820995


In [23]:
sum(gp_cvar[1:])

0.038170581691953906